In [ ]:
import pandas as pd
import numpy as np
import panel as pn
pn.extension('tabulator') 
# More on this here: https://panel.holoviz.org/reference/widgets/Tabulator.html

import hvplot.pandas
import os

In [ ]:
# Grab some data from Our World in Data. Go to https://github.com/owid/co2-data/blob/master/owid-co2-data.csv and grab the raw data.
df = pd.read_csv('https://raw.githubusercontent.com/owid/co2-data/master/owid-co2-data.csv')

In [ ]:
df

In [ ]:
df.columns

In [ ]:
df[df['country'] == 'World']

# Data Clean Up

In [ ]:
# Fill NAs with 0s and create GDP per capita column
df = df.fillna(0)
df['gdp_per_capita'] = np.where(df['population']!=0, df['gdp']/df['population'],0)

In [ ]:
# Make dataframe pipeline interactive
idf = df.interactive()

# CO2 emission over time per continent

In [ ]:
# Define the Panel widgets
# Find out more widgets here: panel.holoviz.org/user_guide/Widgets.html
year_slider = pn.widgets.IntSlider(name='Year Slider', start=1750, end=2020, step=5, value=1850)
year_slider

In [ ]:
# Radio buttons for CO2 measures
yaxis_co2 = pn.widgets.RadioButtonGroup(
    name = 'Y axis',
    options = ['co2', 'co2_per_capita',],
    button_type = 'success'
)

In [ ]:
# Let's create a CO2 Data Pipeline
continents = ['World', 'Asia', 'Oceania', 'Europe', 'Africa', 'North America', 'South America', 'Antarctica' ]

co2_pipeline = (
    idf[
        (idf.year <= year_slider) &
        (idf.country.isin(continents))
    ]
    .groupby(['country', 'year'])[yaxis_co2].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='year')
    .reset_index(drop=True)
)

In [ ]:
# Here is what the data pipeline looks like
co2_pipeline.head(n=10)

In [ ]:
# Here is a plot of the data pipeline
co2_plot = co2_pipeline.hvplot(x = 'year', by = 'country', y=yaxis_co2, line_width=2, title = 'CO2 Emission by Continent')
co2_plot

# Table of CO2 Emission Over Time By Continent

In [ ]:
co2_table = co2_pipeline.pipe(pn.widgets.Tabulator, pagination='remote', page_size = 10, sizing_mode='stretch_width')
co2_table

In [ ]:
# Let's build a new timeline for CO2 vs. GDP. 
# We will look at countries and not continents this time.

co2_vs_gdp_scatterplot_pipeline = (
    idf[
        (idf.year == year_slider) &
        (~ (idf.country.isin(continents)))
    ]
    .groupby(['country', 'year', 'gdp_per_capita'])['co2'].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='year')
    .reset_index(drop=True)
)

In [ ]:
# Quick preview of the head of the pipeline
co2_vs_gdp_scatterplot_pipeline.head(n=15)

In [ ]:
# Now let's build the scatterplot that's based on the pipeline above
co2_vs_gdp_scatterplot = co2_vs_gdp_scatterplot_pipeline.hvplot(x = 'gdp_per_capita', y = 'co2', by = 'country', size = 80, kind = 'scatter', alpha = 0.7, legend = False, height = 500, width = 500)

# View the scatterplot
co2_vs_gdp_scatterplot

# Bar Chart with CO2 Sources by Continent

In [ ]:
yaxis_co2_source = pn.widgets.RadioButtonGroup(
    name = 'Y axis',
    options = ['coal_co2', 'oil_co2', 'gas_co2'],
    button_type = 'success'
)

continents_excl_world = ['Asia', 'Oceania', 'Europe', 'Africa', 'North America', 'South America', 'Antarctica' ]
co2_source_bar_pipeline = (
    idf[
        (idf.year == year_slider) &
        (idf.country.isin(continents_excl_world))
    ]
    .groupby(['year', 'country'])[yaxis_co2_source].sum()
    .to_frame()
    .reset_index()
    .sort_values(by='year')
    .reset_index(drop=True)
)

In [ ]:
co2_source_bar_plot = co2_source_bar_pipeline.hvplot(kind = 'bar', x = 'country',xlabel = 'Continents', y = yaxis_co2_source, title = 'CO2 Source by Continent')
co2_source_bar_plot

# Let's Create the Dashboard

In [ ]:
# Layout using the FastListTemplate
# More on the template here: https://panel.holoviz.org/reference/templates/FastListTemplate.html

# Get the directory of the current Jupyter Notebook
notebook_dir = os.path.dirname(os.path.abspath("__file__"))

# Construct the path to your image relative to the notebook's directory
image_path = os.path.join(notebook_dir, "co2_dashboard_practice/img/randomimage.png")

fast_list_template = pn.template.FastListTemplate(
    title = 'CO2 Emission Dashboard based on Data from Our World in Data',
    sidebar = [
        pn.pane.Markdown("# CO2 Emissions and Climate Change"),
        pn.pane.Markdown("#### Carbon dioxide emissions are believed to be the primary drivers of global climate change."),
        pn.pane.Markdown("## Simple Image"),
        pn.pane.PNG(image_path, sizing_mode = 'scale_both'),
        pn.pane.Markdown("## Settings"),
        year_slider],
    main = [
        pn.Row(pn.Column(yaxis_co2, co2_plot.panel(width=700), margin=(0,25)), 
               co2_table.panel(width=500)),
        pn.Row(pn.Column(co2_vs_gdp_scatterplot.panel(width=600), margin=(0,25)),
              pn.Column(yaxis_co2_source, co2_source_bar_plot.panel(width=600)))
    ],
    accent_base_color = "#88d8b0",
    header_background = "#88d8b0",
)

# Time to display the dashboard
fast_list_template.servable();

# Launch the Dashboard via Terminal

At this point in the process, go to the terminal. Preferably in the same virtual environment as this notebook. If possible, use the terminal console in this instance of Jupyter Lab where the notebook is live. After typing "panel serve thisnotebookname.ipynb" in the terminal, there should come a URL in the terminal that you can then copy and paste in a browser to vide the dashboards.